In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install catboost
!pip3 install category_encoders
!pip install -U scikit-learn
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.3 MB/s eta 0:00:00


In [3]:
import os
import datetime
import numpy as np
import pandas as pd
import unicodedata
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder, KBinsDiscretizer, RobustScaler, OneHotEncoder
)
from category_encoders import CountEncoder
import optuna
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder
import warnings
warnings.simplefilter("ignore")
from sklearn.decomposition import TruncatedSVD

In [4]:
# Constants
exp_name = 'exp00049'
INPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/signate2023/")
OUTPUT_PATH = f'/content/drive/MyDrive/Colab Notebooks/signate2023/{exp_name}/'
SUBMISSION_DIR = os.path.join(OUTPUT_PATH, 'submissions')
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, f'submission_{exp_name}.csv')
MODEL_DIR = os.path.join(OUTPUT_PATH, 'signate-models')
os.makedirs(MODEL_DIR, exist_ok=True)

# Load Data
train_df = pd.read_csv(INPUT_DIR / "train.csv")
test_df = pd.read_csv(INPUT_DIR / "test.csv")

In [5]:
state_dic = {}

# データ数的に、重複は気にせず上書きしていっても大した時間にはならない
for i in train_df.index:
    # ただし、欠損値で上書きされると困るので、そこだけケア
    if pd.isna(train_df.at[i, 'state'])==False:
        state_dic[train_df.at[i, 'region']] = train_df.at[i, 'state']

train_df.loc[pd.isna(train_df['state']).index, 'state'] = train_df.loc[pd.isna(train_df['state']).index, 'region'].map(state_dic)
test_df.loc[pd.isna(test_df['state']).index, 'state'] = test_df.loc[pd.isna(test_df['state']).index, 'region'].map(state_dic)

train_df.drop(['region'], axis=1, inplace=True)
test_df.drop(['region'], axis=1, inplace=True)

In [6]:
# Define a function to normalize manufacturer names
def normalize_manufacturer_name(name):
    # Convert full-width characters to half-width
    name = unicodedata.normalize('NFKC', name)
    # Convert to lowercase
    name = name.lower()
    # Remove non-ASCII characters
    name = ''.join(char for char in name if ord(char) < 128)
    return name


#fitしてtransformしないものはここで処理
def preprocessing(df):
    df[["year", "odometer"]] = df[["year", "odometer"]].astype("float32")

    df["cylinders"] = df["cylinders"].astype(str).str.extract("(\d+)").astype("float32")

    # sizeの表記揺れを修正
    df["size"] = df["size"].str.replace("ー", "-").astype(str)
    df["size"] = df["size"].str.replace("−", "-").astype(str)

    # Apply the function to the 'manufacturer' column
    df['manufacturer'] = df['manufacturer'].apply(normalize_manufacturer_name)

    df.loc[df['odometer'] == -1, 'odometer'] = np.nan


    # Define a dictionary for manufacturer name mapping with special characters
    manufacturer_name_mapping = {
        'nisan': 'nissan',
        'toyot': 'toyota',
        'nisѕan': 'nissan',
        'subαru': 'subaru',
        'subru': 'subaru',
        'sαturn': 'saturn',
        'lexu':'lexus',
        'аcura': 'acura',
        'vоlkswagen': 'volkswagen',
        'vlkswagen': 'volkswagen',
        'lexuѕ': 'lexus',
        'ᴄhrysler': 'chrysler',
        'sturn': 'saturn'
    }

    # Replace the manufacturer names using the mapping
    df['manufacturer'] = df['manufacturer'].replace(manufacturer_name_mapping)


    year_dict = {
        2999:1999,
        3008:2008,
        3011:2011,
        3015:2015,
        3017:2017,
        3019:2019,
    }
    df["year"] = df["year"].replace(year_dict)

    df["elapsed_years"] = 2023 - df["year"]

    # 走行距離 / 製造年
    df["odometer_per_year"] = df["odometer"] / (2023 - df["year"])

    df['odometer_log'] = np.log1p(df['odometer'])

    # Create a new feature 'car_age'
    current_year = datetime.datetime.now().year
    df['car_age'] = current_year - df['year']

    # odometer_scaledとcar_ageの交互作用を新たな特徴量として作成します
    df['odometer_age_interaction'] = df['odometer'] * df['car_age']

    # 2. 'odometer'の時間経過による変化
    df['odometer_per_car_age'] = df['odometer'] / df['car_age']

    # 5. 'odometer'のビン化
    df['odometer_bin_100k'] = pd.cut(df['odometer'], bins=[0, 100000, 200000, 300000, np.inf], labels=False)

    # 6. 'odometer'の他の数学的変換
    df['odometer_sqrt'] = np.sqrt(df['odometer'])

    return df

In [7]:
train_feat_df = preprocessing(train_df)
test_feat_df = preprocessing(test_df)

In [8]:
pd.set_option('display.max_columns', None)
train_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,elapsed_years,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,0,1949.0,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,tn,27587,74.0,1556.054077,11.653982,74.0,8520952.0,1556.054077,1.0,339.334656
1,1,2013.0,toyota,fair,8.0,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,4724,10.0,17203.800781,12.055476,10.0,1720380.0,17203.800781,1.0,414.774628
2,2,1998.0,ford,good,6.0,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,10931,25.0,6099.680176,11.934874,25.0,3812300.0,6099.680176,1.0,390.502228
3,3,2014.0,ford,excellent,4.0,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,16553,9.0,11568.666992,11.553289,9.0,937062.0,11568.666992,1.0,322.673218
4,4,2005.0,ford,excellent,6.0,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,5158,18.0,8030.777832,11.881415,18.0,2601972.0,8030.777832,1.0,380.202576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008.0,ford,good,6.0,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,32212,15.0,1777.333374,10.190957,15.0,399900.0,1777.333374,0.0,163.278900
27528,27528,2007.0,ford,excellent,8.0,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,ok,5400,16.0,6754.500000,11.590562,16.0,1729152.0,6754.500000,1.0,328.743073
27529,27529,2019.0,jeep,like new,6.0,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,22227,4.0,34977.000000,11.848747,4.0,559632.0,34977.000000,1.0,374.042786
27530,27530,2007.0,jeep,excellent,6.0,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3054,16.0,7020.375000,11.629169,16.0,1797216.0,7020.375000,1.0,335.150726


In [9]:
def get_custom_cv(df, n_splits=5):
    # priceを小さい順に各foldに振り分ける
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)


    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

n_splits = 8
cv = list(get_custom_cv(train_feat_df, n_splits))

The variance of the mean of the folds:  4.695908800203918
0    0
1    3
2    0
3    2
4    3
5    3
6    3
7    3
8    0
9    4
Name: fold, dtype: int64
27522    4
27523    2
27524    2
27525    0
27526    0
27527    2
27528    1
27529    0
27530    1
27531    4
Name: fold, dtype: int64

In [10]:
train_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,elapsed_years,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,0,1949.0,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,tn,27587,74.0,1556.054077,11.653982,74.0,8520952.0,1556.054077,1.0,339.334656
1,1,2013.0,toyota,fair,8.0,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,4724,10.0,17203.800781,12.055476,10.0,1720380.0,17203.800781,1.0,414.774628
2,2,1998.0,ford,good,6.0,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,10931,25.0,6099.680176,11.934874,25.0,3812300.0,6099.680176,1.0,390.502228
3,3,2014.0,ford,excellent,4.0,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,16553,9.0,11568.666992,11.553289,9.0,937062.0,11568.666992,1.0,322.673218
4,4,2005.0,ford,excellent,6.0,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,5158,18.0,8030.777832,11.881415,18.0,2601972.0,8030.777832,1.0,380.202576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008.0,ford,good,6.0,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,32212,15.0,1777.333374,10.190957,15.0,399900.0,1777.333374,0.0,163.278900
27528,27528,2007.0,ford,excellent,8.0,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,ok,5400,16.0,6754.500000,11.590562,16.0,1729152.0,6754.500000,1.0,328.743073
27529,27529,2019.0,jeep,like new,6.0,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,22227,4.0,34977.000000,11.848747,4.0,559632.0,34977.000000,1.0,374.042786
27530,27530,2007.0,jeep,excellent,6.0,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3054,16.0,7020.375000,11.629169,16.0,1797216.0,7020.375000,1.0,335.150726


In [11]:
test_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,elapsed_years,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,27532,2015.0,chevrolet,excellent,4.0,gas,92553.0,clean,automatic,fwd,full-size,SUV,red,co,8.0,11569.125000,11.435548,8.0,740424.0,11569.125000,0.0,304.225250
1,27533,2013.0,nissan,like new,4.0,gas,134385.0,salvage,automatic,fwd,mid-size,sedan,black,or,10.0,13438.500000,11.808472,10.0,1343850.0,13438.500000,1.0,366.585602
2,27534,2011.0,volkswagen,good,4.0,gas,102489.0,clean,automatic,fwd,full-size,sedan,black,oh,12.0,8540.750000,11.537520,12.0,1229868.0,8540.750000,1.0,320.139038
3,27535,2016.0,jeep,excellent,6.0,diesel,64310.0,clean,automatic,4wd,mid-size,SUV,red,co,7.0,9187.142578,11.071486,7.0,450170.0,9187.142578,0.0,253.594162
4,27536,1999.0,honda,excellent,8.0,gas,180839.0,rebuilt,automatic,4wd,mid-size,SUV,silver,nc,24.0,7534.958496,12.105368,24.0,4340136.0,7534.958496,1.0,425.251678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,2016.0,gmc,excellent,6.0,gas,90902.0,rebuilt,automatic,fwd,full-size,truck,black,mt,7.0,12986.000000,11.417548,7.0,636314.0,12986.000000,0.0,301.499573
27533,55065,2012.0,nissan,excellent,4.0,gas,27234.0,rebuilt,automatic,fwd,mid-size,sedan,white,nv,11.0,2475.818115,10.212258,11.0,299574.0,2475.818115,0.0,165.027267
27534,55066,2002.0,bmw,excellent,6.0,gas,99761.0,clean,automatic,rwd,full-size,coupe,blue,az,21.0,4750.523926,11.510543,21.0,2094981.0,4750.523926,0.0,315.849640
27535,55067,2006.0,bmw,excellent,6.0,gas,162279.0,clean,automatic,4wd,mid-size,sedan,silver,co,17.0,9545.823242,11.997079,17.0,2758743.0,9545.823242,1.0,402.838684


In [12]:
def preprocessing_per_fold(X_tr, y_tr, X_val, transformed_test_data, te_cols):


    scaler = RobustScaler()
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
    imputer = SimpleImputer(strategy='median')

    # RobustScaler
    X_tr['odometer_scaled'] = scaler.fit_transform(X_tr[['odometer']])
    X_val['odometer_scaled'] = scaler.transform(X_val[['odometer']])
    transformed_test_data['odometer_scaled'] = scaler.transform(transformed_test_data[['odometer']])

    # Binning
    X_tr['odometer_scaled_bin'] = pd.cut(X_tr['odometer'], bins=10, labels=False)
    X_val['odometer_scaled_bin'] = pd.cut(X_val['odometer'], bins=10, labels=False)
    transformed_test_data['odometer_scaled_bin'] = pd.cut(transformed_test_data['odometer'], bins=10, labels=False)

    X_tr['car_age_bin'] = pd.cut(X_tr['car_age'], bins=10, labels=False)
    X_val['car_age_bin'] = pd.cut(X_val['car_age'], bins=10, labels=False)
    transformed_test_data['car_age_bin'] = pd.cut(transformed_test_data['car_age'], bins=10, labels=False)



    # One-Hot Encoding for categorical columns
    onehot_cols_standard = ['condition', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color']  # 通常のワンホットエンコーディングのみを行う特徴量リスト
    onehot_cols_svd = ['manufacturer', 'state']  # ワンホットエンコーディング後に次元削減を行う特徴量リスト

    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

    # Standard One-Hot Encoding
    ohe.fit(X_tr[onehot_cols_standard])
    columns_ohe_standard = ohe.get_feature_names_out(onehot_cols_standard)
    X_ohe_tr_standard = ohe.transform(X_tr[onehot_cols_standard])
    X_ohe_val_standard = ohe.transform(X_val[onehot_cols_standard])
    test_ohe_standard = ohe.transform(transformed_test_data[onehot_cols_standard])
    X_tr = pd.concat([X_tr, pd.DataFrame(X_ohe_tr_standard, columns=columns_ohe_standard, index=X_tr.index)], axis=1)
    X_val = pd.concat([X_val, pd.DataFrame(X_ohe_val_standard, columns=columns_ohe_standard, index=X_val.index)], axis=1)
    transformed_test_data = pd.concat([transformed_test_data, pd.DataFrame(test_ohe_standard, columns=columns_ohe_standard, index=transformed_test_data.index)], axis=1)

    # One-Hot Encoding with SVD
    SVD_DIM = 5
    SEED = 10
    for col in onehot_cols_svd:
        # ワンホットエンコーディング
        ohe_individual = OneHotEncoder(sparse=False, handle_unknown='ignore')
        ohe_individual.fit(X_tr[[col]])
        X_ohe_tr_individual = ohe_individual.transform(X_tr[[col]])
        X_ohe_val_individual = ohe_individual.transform(X_val[[col]])
        test_ohe_individual = ohe_individual.transform(transformed_test_data[[col]])

        # TruncatedSVDを用いて次元圧縮
        SVD = TruncatedSVD(n_components=SVD_DIM, random_state=SEED)
        X_tr_svd = SVD.fit_transform(X_ohe_tr_individual)
        X_val_svd = SVD.transform(X_ohe_val_individual)
        test_svd = SVD.transform(test_ohe_individual)

        # 圧縮した特徴量を元のデータフレームに結合
        X_tr = pd.concat([X_tr, pd.DataFrame(X_tr_svd, columns=[f'svd_{col}_{i}' for i in range(1, SVD_DIM+1)], index=X_tr.index)], axis=1)
        X_val = pd.concat([X_val, pd.DataFrame(X_val_svd, columns=[f'svd_{col}_{i}' for i in range(1, SVD_DIM+1)], index=X_val.index)], axis=1)
        transformed_test_data = pd.concat([transformed_test_data, pd.DataFrame(test_svd, columns=[f'svd_{col}_{i}' for i in range(1, SVD_DIM+1)], index=transformed_test_data.index)], axis=1)



    #クラスタリング
    # SimpleImputerをインスタンス化（ここでは欠損値を中央値で補完）
    imputer = SimpleImputer(strategy='median')
    # 補完を行いたい特徴量を選択
    features = X_tr[['odometer', 'car_age']].copy()
    # 欠損値補完
    features_imputed = imputer.fit_transform(features)
    # KMeansクラスタリングを設定
    kmeans = KMeans(n_clusters=10, random_state=0)
    # 補完されたデータに基づいてクラスタリング
    X_tr['odometer_age_cluster'] = kmeans.fit_predict(features_imputed)
    # 検証データについても同様に処理
    features_val = X_val[['odometer', 'car_age']].copy()
    features_val_imputed = imputer.transform(features_val)
    X_val['odometer_age_cluster'] = kmeans.predict(features_val_imputed)
    # テストデータについても同様に処理
    features_test = transformed_test_data[['odometer', 'car_age']].copy()
    features_test_imputed = imputer.transform(features_test)
    transformed_test_data['odometer_age_cluster'] = kmeans.predict(features_test_imputed)
    # 元のodometer_scaledの欠損値を復元
    X_tr['odometer'] = features['odometer']
    X_val['odometer'] = features_val['odometer']
    transformed_test_data['odometer'] = features_test['odometer']


    # 1. odometerに対する他の特徴量の比率
    mean_odometer_per_year = X_tr.groupby('year')['odometer'].transform('mean')
    X_tr['odometer_ratio_year'] = X_tr['odometer'] / mean_odometer_per_year
    X_val['odometer_ratio_year'] = X_val['odometer'] / mean_odometer_per_year
    transformed_test_data['odometer_ratio_year'] = transformed_test_data['odometer'] / mean_odometer_per_year

    # Count encoding
    categorical_features = ["cylinders", "manufacturer", "condition", "fuel", "title_status", "transmission", "drive", "size", "type", "paint_color", "state",]
    for col in categorical_features:
        count_map = X_tr[col].value_counts().to_dict()
        X_tr[col + '_count'] = X_tr[col].map(count_map)
        X_val[col + '_count'] = X_val[col].map(count_map)
        transformed_test_data[col + '_count'] = transformed_test_data[col].map(count_map)

    # 集約特徴量を用いたエンコーディング
    for col in categorical_features:
        for agg_ in ["mean", "std", "max", "min", "median"]:
            fillna_map = X_tr.groupby(col)["price"].agg(agg_)
            X_tr[col+f"_{agg_}_encoding"] = X_tr[col].map(fillna_map)
            X_val[col+f"_{agg_}_encoding"] = X_val[col].map(fillna_map)
            transformed_test_data[col+f"_{agg_}_encoding"] = transformed_test_data[col].map(fillna_map)

    # 集約特徴量を用いたエンコーディング
    for col in categorical_features:
        for agg_ in ["mean", "std", "max", "min", "median"]:
            fillna_map = X_tr.groupby(col)["odometer"].agg(agg_)
            X_tr[col+f"_{agg_}_odometer_encoding"] = X_tr[col].map(fillna_map)
            X_val[col+f"_{agg_}_odometer_encoding"] = X_val[col].map(fillna_map)
            transformed_test_data[col+f"_{agg_}_odometer_encoding"] = transformed_test_data[col].map(fillna_map)
            if agg_ == "median" or agg_ == "mean":
                X_tr[col+f"_{agg_}_odometer_encoding_diff"] = X_tr[col+f"_{agg_}_odometer_encoding"] - X_tr["odometer"]
                X_val[col+f"_{agg_}_odometer_encoding_diff"] = X_val[col+f"_{agg_}_odometer_encoding"] - X_val["odometer"]
                transformed_test_data[col+f"_{agg_}_odometer_encoding_diff"] = transformed_test_data[col+f"_{agg_}_odometer_encoding"] - transformed_test_data["odometer"]

    # 集約特徴量を用いたエンコーディング
    for col in categorical_features:
        for agg_ in ["mean", "std", "max", "min", "median"]:
            fillna_map = X_tr.groupby(col)["elapsed_years"].agg(agg_)
            X_tr[col+f"_{agg_}_elapsed_years_encoding"] = X_tr[col].map(fillna_map)
            X_val[col+f"_{agg_}_elapsed_years_encoding"] = X_val[col].map(fillna_map)
            transformed_test_data[col+f"_{agg_}_elapsed_years_encoding"] = transformed_test_data[col].map(fillna_map)
            if agg_ == "median" or agg_ == "mean":
                X_tr[col+f"_{agg_}_elapsed_years_encoding_diff"] = X_tr[col+f"_{agg_}_elapsed_years_encoding"] - X_tr["elapsed_years"]
                X_val[col+f"_{agg_}_elapsed_years_encoding_diff"] = X_val[col+f"_{agg_}_elapsed_years_encoding"] - X_val["elapsed_years"]
                transformed_test_data[col+f"_{agg_}_elapsed_years_encoding_diff"] = transformed_test_data[col+f"_{agg_}_elapsed_years_encoding"] - transformed_test_data["elapsed_years"]


    #2変数ターゲットエンコーディング
    X_tr["year_map"], bins = pd.cut(X_tr["year"], bins=20, labels=False, retbins=True)
    X_val["year_map"] = pd.cut(X_val["year"], bins=bins, labels=False)
    transformed_test_data["year_map"] = pd.cut(transformed_test_data["year"], bins=bins, labels=False)
    X_tr["odometer_map"], bins = pd.cut(X_tr["odometer"], bins=20, labels=False, retbins=True)
    X_val["odometer_map"] = pd.cut(X_val["odometer"], bins=bins, labels=False)
    transformed_test_data["odometer_map"] = pd.cut(transformed_test_data["odometer"], bins=bins, labels=False)


    cross_features = [
        'year_map', 'manufacturer', 'condition', 'cylinders','fuel', 'odometer_map', 'title_status', 'transmission', 'drive', 'size',
        'type', 'paint_color', 'state', 'odometer_age_cluster'
    ]

    for i, col1 in enumerate(cross_features):
        for col2 in cross_features[i+1:]:
            tmp = X_tr.groupby([col1, col2])["price"].mean().reset_index()
            X_tr = pd.merge(X_tr, tmp.rename(columns={"price":f"{col1}*{col2}_price"}), on=[col1, col2], how="left")
            X_val = pd.merge(X_val, tmp.rename(columns={"price":f"{col1}*{col2}_price"}), on=[col1, col2], how="left")
            transformed_test_data = pd.merge(transformed_test_data, tmp.rename(columns={"price":f"{col1}*{col2}_price"}), on=[col1, col2], how="left")


    return X_tr, X_val, transformed_test_data


In [13]:
def fit_lgbm(X, y, test_data, cv, te_cols, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    target_encoders = []
    transformed_test_data_list = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]
        transformed_test_data = test_data.copy()

        X_tr, X_val, transformed_test_data = preprocessing_per_fold(X_tr, y_tr, X_val, transformed_test_data, te_cols)

        drop_cols = ['id', 'manufacturer', 'condition', 'fuel', 'title_status',
                     'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
        X_tr.drop(drop_cols+['price'], axis=1, inplace=True)
        X_val.drop(drop_cols+['price'], axis=1, inplace=True)
        transformed_test_data.drop(drop_cols, axis=1, inplace=True)


        transformed_test_data_list.append(transformed_test_data)


        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, transformed_test_data_list




# optunaを使ったパラメータチューニング
def tuning(train_feat_df, train_df, cv):
     def objective(trial):
         max_depth = trial.suggest_int("max_depth", 1, 10)
         params = {
              "max_depth": max_depth,
              "n_estimators": trial.suggest_int('n_estimators', 100, 50000),
              "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
              "num_leaves": trial.suggest_int("num_leaves", 2, 2**max_depth),
              "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
              "subsample": trial.suggest_float("subsample", 0.01, 1.0),
              "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
              "max_bin": trial.suggest_int("max_bin", 32, 512),
              "reg_lambda": trial.suggest_float("reg_lambda", 1e-10, 100.0),
              "task":"train",
              "objective": "mape",
              "boosting":"gbdt",
              "metric": 'mape',
              "importance_type": "gain",
              "seed" : 88,
              "verbosity": -1,


         }

         te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

         score, _, _, _ = fit_lgbm(
             X=train_feat_df, y=train_df["price"], test_data=test_feat_df, cv=cv, te_cols=te_cols,  categorical_cols=[], params=params, verbose=-1
         )
         return score

     study = optuna.create_study(direction="minimize")
     study.optimize(objective, n_trials=100)
     print("Number of finished trials:", len(study.trials))
     print("Best trial:", study.best_trial.params)
     return study.best_trial.params

In [ ]:
{'max_depth': 10, 'n_estimators': 37725, 'learning_rate': 0.010908099604541203, 'num_leaves': 319, 'colsample_bytree': 0.20764018910087123, 'subsample': 0.8803116946706183, 'min_data_in_leaf': 235, 'max_bin': 43, 'reg_lambda': 13.674907094349944}
FINISHI: Whole Score: 0.44002

In [14]:
lgbm_params = {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "importance_type": "gain",
    "random_state": 88,
    "verbose": -1,
    'max_depth': 10,
    'n_estimators': 37725,
    'learning_rate': 0.010908099604541203,
    'num_leaves': 319,
    'colsample_bytree': 0.20764018910087123,
    'subsample': 0.8803116946706183,
    'min_data_in_leaf': 235,
    'max_bin': 43,
    'reg_lambda': 13.674907094349944
}

te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

# training
score, oof_lgb, models_lgb, transformed_test_data_list  = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    te_cols=te_cols,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

 - fold1 - 0.4373
 - fold2 - 0.4472
 - fold3 - 0.4358
 - fold4 - 0.4506
 - fold5 - 0.4337
 - fold6 - 0.4307
 - fold7 - 0.4477
 - fold8 - 0.4390
FINISHI: Whole Score: 0.4402


In [ ]:
lgbm_params = tuning(train_feat_df, train_df, cv)

lgbm_params.update(
    {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "importance_type": "gain",
    "seed" : 88,
    "verbosity": -1,
    }
)

te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    te_cols=te_cols,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

[I 2023-08-20 05:01:32,311] A new study created in memory with name: no-name-c28999dd-e6e8-48b3-befe-d159e3cc000d


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[838]	valid_0's mape: 0.447693
 - fold1 - 0.4477
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[794]	valid_0's mape: 0.46009
 - fold2 - 0.4601
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[171]	valid_0's mape: 0.449327
 - fold3 - 0.4493
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[776]	valid_0's mape: 0.461798
 - fold4 - 0.4618
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[903]	valid_0's mape: 0.443083
 - fold5 - 0.4431
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[838]	valid_0's mape: 0.441206
 - fold6 - 0.4412
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[362]	valid_0's mape: 0.4590

[I 2023-08-20 05:06:39,639] Trial 0 finished with value: 0.4517869705365331 and parameters: {'max_depth': 3, 'n_estimators': 43081, 'learning_rate': 0.3918852953551303, 'num_leaves': 2, 'colsample_bytree': 0.5999779901013272, 'subsample': 0.29192947424467164, 'min_data_in_leaf': 24, 'max_bin': 361, 'reg_lambda': 15.499315900452316}. Best is trial 0 with value: 0.4517869705365331.


Early stopping, best iteration is:
[368]	valid_0's mape: 0.452078
 - fold8 - 0.4521
FINISHI: Whole Score: 0.4518
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[13]	valid_0's mape: 0.450426
 - fold1 - 0.4504
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[63]	valid_0's mape: 0.460939
 - fold2 - 0.4609
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[13]	valid_0's mape: 0.453893
 - fold3 - 0.4539
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11]	valid_0's mape: 0.466946
 - fold4 - 0.4669
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's mape: 0.453358
 - fold5 - 0.4534
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[20]	valid_0's mape: 0.448513
 - fold6 - 0.4485
Training until val

[I 2023-08-20 05:10:15,811] Trial 1 finished with value: 0.45664170132031234 and parameters: {'max_depth': 5, 'n_estimators': 33874, 'learning_rate': 0.6488618147553623, 'num_leaves': 13, 'colsample_bytree': 0.39888272885497544, 'subsample': 0.06078139085246764, 'min_data_in_leaf': 221, 'max_bin': 310, 'reg_lambda': 54.53252555619856}. Best is trial 0 with value: 0.4517869705365331.


...

Early stopping, best iteration is:
[182]	valid_0's mape: 0.441318
 - fold8 - 0.4413
FINISHI: Whole Score: 0.4415
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[88]	valid_0's mape: 0.439558
 - fold1 - 0.4396
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[123]	valid_0's mape: 0.449766
 - fold2 - 0.4498
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[54]	valid_0's mape: 0.438722
 - fold3 - 0.4387
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[48]	valid_0's mape: 0.450193
 - fold4 - 0.4502
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[52]	valid_0's mape: 0.435634
 - fold5 - 0.4356
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[141]	valid_0's mape: 0.434832
 - fold6 - 0.4348
Training until 

[I 2023-08-20 11:22:37,133] Trial 89 finished with value: 0.4429505165631562 and parameters: {'max_depth': 10, 'n_estimators': 35818, 'learning_rate': 0.0918452675099098, 'num_leaves': 419, 'colsample_bytree': 0.21555087641793858, 'subsample': 0.8113670272836383, 'min_data_in_leaf': 234, 'max_bin': 33, 'reg_lambda': 28.18975250850396}. Best is trial 69 with value: 0.440247966635569.


Early stopping, best iteration is:
[37]	valid_0's mape: 0.441866
 - fold8 - 0.4419
FINISHI: Whole Score: 0.4430
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[175]	valid_0's mape: 0.43787
 - fold1 - 0.4379
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[181]	valid_0's mape: 0.449585
 - fold2 - 0.4496
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[159]	valid_0's mape: 0.437666
 - fold3 - 0.4377
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[125]	valid_0's mape: 0.450629
 - fold4 - 0.4506
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[228]	valid_0's mape: 0.434282
 - fold5 - 0.4343
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[184]	valid_0's mape: 0.432248
 - fold6 - 0.4322
Training unti

[I 2023-08-20 11:26:47,332] Trial 90 finished with value: 0.44131268415796726 and parameters: {'max_depth': 9, 'n_estimators': 26653, 'learning_rate': 0.03503387584160207, 'num_leaves': 337, 'colsample_bytree': 0.30150666249453606, 'subsample': 0.9046074158767293, 'min_data_in_leaf': 221, 'max_bin': 88, 'reg_lambda': 19.851555126804055}. Best is trial 69 with value: 0.440247966635569.


Early stopping, best iteration is:
[155]	valid_0's mape: 0.440454
 - fold8 - 0.4405
FINISHI: Whole Score: 0.4413


In [ ]:
{'max_depth': 10, 'n_estimators': 37725, 'learning_rate': 0.010908099604541203, 'num_leaves': 319, 'colsample_bytree': 0.20764018910087123, 'subsample': 0.8803116946706183, 'min_data_in_leaf': 235, 'max_bin': 43, 'reg_lambda': 13.674907094349944}
FINISHI: Whole Score: 0.44002

In [ ]:
# Create a copy of the dataframe
#train_data_transformed = train_feat_df.copy()#

# Apply target encoding to the copy of the dataframe
#for col in te_cols:
#    # Create a new instance of TargetEncoder and fit it to the whole data
#    te = TargetEncoder(target_type="continuous", random_state=88)
#    te.fit(train_data_transformed[[col]], train_df["price"])
#    train_data_transformed['te_' + col] = te.transform(train_data_transformed[[col]])

## Drop the original columns
#drop_cols = ['ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status',
#       'ori__transmission', 'ori__drive', 'ori__size', 'ori__type', 'ori__paint_color', 'ori__state']
#train_data_transformed.drop(drop_cols, axis=1, inplace=True)

## Check the feature importances
#importances = pd.DataFrame({
#    'Feature': train_data_transformed.columns,
##    'Importance': models_lgb[0].feature_importances_
#}).sort_values(by='Importance', ascending=False)
#importances


In [15]:
sub_df = pd.read_csv(INPUT_DIR / "submit_sample.csv", names=["id", "price"])

In [17]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{exp_name}.csv', index=False, header=False)


In [23]:
pd.concat([train_df['id'], pd.DataFrame(oof_lgb, columns=['oof_pred'])], axis=1).to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{exp_name}_oof_pred.csv', index=False)